In [1]:
import matplotlib.pyplot as plt

#### Задача
Подумай пока над протоколом для вот такой задачи.
Есть 10 медальонов. Условно говоря, люди с медальонами шарахаются по лесу, время от времени сближаясь на расстояние, достаточное для прохождения сигнала.
Нужно сделать так, чтобы каждый медальон знал, сколько вокруг него других медальонов.

Если два медальона передают одновременно, то случается коллизия, и они друг друга глушат. То есть их никто не слышит.

Так что нужно придумать, как избежать коллизий. Состояние требуется обновлять раз в четыре секунды

#### Протокол передачи данных
**Отправка:** Пусть каждый медальон отправляет пакет со своим ID с частотой X Гц
**Сбор:** Пусть каждый медальон раз в четыре секунды переходит в состояние RX на $2\cdot\frac{1}{X}$

Корректность: Допустим, что каждый медальон выбирает в четырех секундах время для сбора случайно. Тогда вероятность того, что никакие два медальона пересекутся по времени передачи примерно равна $\prod\limits_{i = 2}^{10}{\left(1 - \frac{ 2\cdot\frac{1}{X}}{4}\cdot i \right)}$

Если никакие два медальона не пересекаются по времени приема, то во время приема одного медальона, остальные передают. Поскольку время RX = 2 * период отправки, то каждый медальон отправит хотя бы два раза сообщение принимающему медальону. Предлагается случайно менять время между двумя передачами, чтобы препятствовать синхронизации отдельных медальонов.

Посчитаем энергопотребление, как функцию от частоты
**MCU (1MHZ):**
1. wake up: 1mcS
2. 1mcA sleep current, 1MhZ = 270mcA

**CC1101:**
1. wake up: 240mcs from sleep to tx/rx
2. 200 nA sleep mode, RX - 15mA, TX - 30mA

Посчитаем потребление за 4с:
X * (240 * 10**(-6) + )

